<a href="https://colab.research.google.com/github/jwbarbona/labo2025v/blob/main/src/workflows/610_WorkFlow_01_gerencial_67295.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


cp: cannot stat '/content/buckets/b1/kaggle/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 05:32:17 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660442,35.3,1454505,77.7,1437489,76.8
Vcells,1226661,9.4,8388608,64.0,1975136,15.1


In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 999983 # 100009, 200003, 499999, 700009, 999983


PARAM$experimento <- 67295
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [ ]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [ ]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [ ]:
# sin codigo en esta primera version del workflow

In [ ]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)


# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.9903030878, 1.9174403544, 1.8296186587,
  1.7728862972, 1.7212488323, 1.6776304408,
  1.6431248196, 1.5814483345, 1.4947526791,
  1.4484037589, 1.3913580777, 1.3404220402,
  1.3154288912, 1.2921698342, 1.2472681797,
  1.2300475145, 1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219, 0.7566381305, 0.7289384687
)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.380952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vUVA <- c(
  2.001408838932958,  1.950325472789153,  1.89323032351521,
  1.8247220405493787, 1.746027787673673,  1.6871348409529485,
  1.6361678865622313, 1.5927529755859773, 1.5549162794128493,
  1.4949100586391746, 1.4197729500774545, 1.3678188186372326,
  1.3136508617223726, 1.2690535173062818, 1.2381595983200178,
  1.211656735577568,  1.1770808941405335, 1.1570338657445522,
  1.1388769475653255, 1.1156993751209352, 1.093638313080772,
  1.0657171590878205, 1.0362173587708712, 1.0,
  0.9669867858358365, 0.9323750098728378, 0.8958202912590305,
  0.8631993702994263, 0.8253893405524657, 0.7928918905364516,
  0.7666323845128089, 0.7428976357662823, 0.721615762047849
)


tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}


drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}


drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}


drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}


# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}


# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$DR$metodo <- "deflacion"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


colnames(dataset)

IPC,dolar_blue,dolar_oficial,UVA,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.9903031,39.04545,38.43000,2.0014088,201901
1.9174404,38.40250,39.42800,1.9503255,201902
1.8296187,41.63947,42.54210,1.8932303,201903
1.7728863,44.27474,44.35421,1.8247220,201904
1.7212488,46.09546,46.08864,1.7460278,201905
1.6776304,45.06333,44.95500,1.6871348,201906
1.6431248,43.98333,43.75143,1.6361679,201907
1.5814483,54.84286,54.65048,1.5927530,201908
1.4947527,61.05952,58.79000,1.5549163,201909


[1] "mrentabilidad"           "mrentabilidad_annual"   
 [3] "mcomisiones"             "mactivos_margen"        
 [5] "mpasivos_margen"         "mcuenta_corriente"      
 [7] "mcaja_ahorro"            "mcuentas_saldo"         
 [9] "mtarjeta_visa_consumo"   "mtarjeta_master_consumo"
[11] "mprestamos_personales"   "mpayroll"               
[13] "Master_mpagominimo"      "Visa_mpagominimo"

inicio drift_deflacion()
fin drift_deflacion()


[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [ ]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [ ]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "kmes"                        "mpayroll_sobre_edad"

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [ ]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 3
dataset[,
    paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 4
dataset[,
    paste0(cols_lagueables, "_lag4") := shift(.SD, 4, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 5
dataset[,
    paste0(cols_lagueables, "_lag5") := shift(.SD, 5, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 6
dataset[,
    paste0(cols_lagueables, "_lag6") := shift(.SD, 6, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}



In [ ]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

cols_promedio <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria","cliente_edad", "cliente_antiguedad", "internet", "cdescubierto_preacordado", "thomebanking",
     "Master_fultimo_cierre", "Master_fechaalta", "Master_status", "Visa_status", "Visa_fultimo_cierre", "Visa_fechaalta")
) )



# Probando EMA

# función general para EMA(N)
emaN <- function(x, n) {
  alpha <- 2 / (n + 1)

  # si todos NA -> devolver NA
  if (all(is.na(x))) return(rep(NA_real_, length(x)))

  # primer EMA = primer valor no NA
  first_idx <- which(!is.na(x))[1]
  ema_vec <- rep(NA_real_, length(x))
  ema_vec[first_idx] <- x[first_idx]

  # recursión
  if (first_idx < length(x)) {
    for (i in (first_idx+1):length(x)) {
      ema_vec[i] <- alpha * x[i] + (1 - alpha) * ema_vec[i - 1]
    }
  }

  return(ema_vec)
}


# aplicar EMA3, EMA6 y EMA12 a todas las columnas de cols_promedio
dataset[
  , c(
      paste0(cols_promedio, "_ema3"),
      paste0(cols_promedio, "_ema6"),
      paste0(cols_promedio, "_ema12")
    ) := c(
      lapply(.SD, function(col) emaN(col, 3)),
      lapply(.SD, function(col) emaN(col, 6)),
      lapply(.SD, function(col) emaN(col, 12))
    ),
  by = numero_de_cliente,
  .SDcols = cols_promedio
]

# creo ratio ult 12 meses vs 6 meses, ult 6 vs ult 3 y actuales vs 3
for (vcol in cols_promedio)
{
    ema6 <- paste0(vcol, "_ema6")
    ema3 <- paste0(vcol, "_ema3")
    ema12 <- paste0(vcol, "_ema12")

    # Ratio EMA3 / EMA6  (aceleración)
    dataset[, paste0(vcol, "ema3_ema6_ratio") :=
              fifelse(get(ema6) == 0 | is.na(get(ema6)),
                      NA_real_, get(ema3) / get(ema6))]

    # Ratio actual / EMA6 - Cambios a mediano plazo
    dataset[, paste0(vcol, "actual_ema6_ratio") :=
              fifelse(get(ema6) == 0 | is.na(get(ema6)),
                      NA_real_, get(vcol) / get(ema6))]


    # Ratio actual / EMA3 - Cambios a corto plazo
    dataset[, paste0(vcol, "actual_ema3_ratio") :=
              fifelse(get(ema3) == 0 | is.na(get(ema3)),
                      NA_real_, get(vcol) / get(ema3))]

    # Ratio actual / EMA12 - Cambios a largo plazo
        dataset[, paste0(vcol, "actual_ema12_ratio") :=
              fifelse(get(ema12) == 0 | is.na(get(ema12)),
                      NA_real_, get(vcol) / get(ema12))]
}

#Elimino emas y dejo solo los ratios
cols_a_borrar <- grep("(_ema3|_ema6|_ema12)$", names(dataset), value = TRUE)
dataset[, (cols_a_borrar) := NULL]

In [ ]:
# Las funciones frollmean y frollapply ya están disponibles a través de data.table,
# que es la librería más eficiente para este tipo de operaciones en R.

# Columnas para Rolling Window (normalmente, monetarias y de transacciones)
cols_rolling <- c("mrentabilidad", "mtarjeta_visa_consumo", "mcuentas_saldo", "cpayroll_trx")
# Puedes usar todas las cols_lagueables, pero las financieras son prioritarias

# Media móvil de los últimos 6 meses (Tendencia)
# frollmean es nativa de data.table y es muy eficiente.
dataset[,
    paste0(cols_rolling, "_mean6") := frollmean(.SD, 6, align = "right", fill = NA, na.rm=TRUE),
    by = numero_de_cliente,
    .SDcols = cols_rolling
]

# Desviación Estándar (SD) móvil de los últimos 6 meses (Volatilidad/Riesgo)
# frollapply con FUN = sd (función base de R) es la alternativa robusta a Rfast.
dataset[,
    paste0(cols_rolling, "_sd6") := frollapply(.SD, 6, FUN = sd, align = "right", fill = NA, na.rm=TRUE),
    by = numero_de_cliente,
    .SDcols = cols_rolling
]

Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

[1] 754

[1] "numero_de_cliente"                                   
  [2] "foto_mes"                                            
  [3] "internet"                                            
  [4] "cliente_edad"                                        
  [5] "cliente_antiguedad"                                  
  [6] "mrentabilidad"                                       
  [7] "mrentabilidad_annual"                                
  [8] "mcomisiones"                                         
  [9] "mactivos_margen"                                     
 [10] "mpasivos_margen"                                     
 [11] "cproductos"                                          
 [12] "mcuenta_corriente"                                   
 [13] "mcaja_ahorro"                                        
 [14] "cdescubierto_preacordado"                            
 [15] "mcuentas_saldo"                                      
 [16] "ctarjeta_visa_transacciones"                         
 [17] "mtarjeta_visa_consumo"                               
 [18] "mtarjeta_master_consumo"                             
 [19] "mprestamos_personales"                               
 [20] "cpayroll_trx"                                        
 [21] "mpayroll"                                            
 [22] "ccomisiones_mantenimiento"                           
 [23] "ccallcenter_transacciones"                           
 [24] "chomebanking_transacciones"                          
 [25] "ctrx_quarter"                                        
 [26] "Master_status"                                       
 [27] "Master_fechaalta"                                    
 [28] "Master_mpagominimo"                                  
 [29] "Visa_status"                                         
 [30] "Visa_fechaalta"                                      
 [31] "Visa_mpagominimo"                                    
 [32] "clase_ternaria"                                      
 [33] "kmes"                                                
 [34] "mpayroll_sobre_edad"                                 
 [35] "internet_lag1"                                       
 [36] "cliente_edad_lag1"                                   
 [37] "cliente_antiguedad_lag1"                             
 [38] "mrentabilidad_lag1"                                  
 [39] "mrentabilidad_annual_lag1"                           
 [40] "mcomisiones_lag1"                                    
 [41] "mactivos_margen_lag1"                                
 [42] "mpasivos_margen_lag1"                                
 [43] "cproductos_lag1"                                     
 [44] "mcuenta_corriente_lag1"                              
 [45] "mcaja_ahorro_lag1"                                   
 [46] "cdescubierto_preacordado_lag1"                       
 [47] "mcuentas_saldo_lag1"                                 
 [48] "ctarjeta_visa_transacciones_lag1"                    
 [49] "mtarjeta_visa_consumo_lag1"                          
 [50] "mtarjeta_master_consumo_lag1"                        
 [51] "mprestamos_personales_lag1"                          
 [52] "cpayroll_trx_lag1"                                   
 [53] "mpayroll_lag1"                                       
 [54] "ccomisiones_mantenimiento_lag1"                      
 [55] "ccallcenter_transacciones_lag1"                      
 [56] "chomebanking_transacciones_lag1"                     
 [57] "ctrx_quarter_lag1"                                   
 [58] "Master_status_lag1"                                  
 [59] "Master_fechaalta_lag1"                               
 [60] "Master_mpagominimo_lag1"                             
 [61] "Visa_status_lag1"                                    
 [62] "Visa_fechaalta_lag1"                                 
 [63] "Visa_mpagominimo_lag1"                               
 [64] "kmes_lag1"                                           
 [65] "mpayroll_sobre_edad_lag1"                            
 [66] "internet_lag2"                

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



In [ ]:
VPOS_CORTE <- c()

fganancia_lgbm_meseta <- function(probs, datos) {
  vlabels <- get_field(datos, "label")
  vpesos <- get_field(datos, "weight")

  tbl <- as.data.table(list(
    "prob" = probs,
    "gan" = ifelse(vlabels == 1 & vpesos > 1, PARAM$CN$train$gan1, PARAM$CN$train$gan0)
  ))

  setorder(tbl, -prob)
  tbl[, posicion := .I]
  tbl[, gan_acum := cumsum(gan)]
  setorder(tbl, -gan_acum) # voy por la meseta

  gan <- mean(tbl[1:500, gan_acum]) # meseta de tamaño 500

  pos_meseta <- tbl[1:500, median(posicion)]
  VPOS_CORTE <<- c(VPOS_CORTE, pos_meseta)

  return(list(
    "name" = "ganancia",
    "value" = gan,
    "higher_better" = TRUE
  ))
}


# Elimina del dataset las variables que estan por debajo
#  de la capa geologica de canaritos
# se llama varias veces, luego de agregar muchas variables nuevas,
#  para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1

campitos <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

CanaritosAsesinos <- function(
  canaritos_ratio,
  canaritos_desvios,
  canaritos_semilla) {

  cat( "inicio CanaritosAsesinos()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$CN$train$clase01_valor1,
      clase01 := 1L ]

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  for (i in 1:(ncol(dataset) * canaritos_ratio)) {
    dataset[, paste0("canarito", i) := runif(nrow(dataset))]
  }

  campos_buenos <- setdiff(
    colnames(dataset),
    c( campitos, "clase01")
  )

  azar <- runif(nrow(dataset))

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$CN$train$training &
      (clase01 == 1 | azar < PARAM$CN$train$undersampling))]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    weight = dataset[
      entrenamiento == TRUE,
      ifelse(clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )

  dvalid <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$CN$train$validation, campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$CN$train$validation, clase01],
    weight = dataset[
      foto_mes %in% PARAM$CN$train$validation,
      ifelse( clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )


  param <- list(
    objective = "binary",
    metric = "custom",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    seed = canaritos_semilla,
    max_depth = -1, # -1 significa no limitar,  por ahora lo dejo fijo
    min_gain_to_split = 0.0, # por ahora, lo dejo fijo
    lambda_l1 = 0.0, # por ahora, lo dejo fijo
    lambda_l2 = 0.0, # por ahora, lo dejo fijo
    max_bin = 31, # por ahora, lo dejo fijo
    num_iterations = 9999, # un numero grande, lo limita early_stopping_rounds
    force_row_wise = TRUE, # para que los alumnos no se atemoricen con  warning
    learning_rate = 0.065,
    feature_fraction = 1.0, # lo seteo en 1
    min_data_in_leaf = 260,
    num_leaves = 60,
    early_stopping_rounds = 200,
    num_threads = 1
  )

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  modelo <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalid),
    eval = fganancia_lgbm_meseta,
    param = param,
    verbose = -100
  )

  tb_importancia <- lgb.importance(model = modelo)
  tb_importancia[, pos := .I]

  fwrite(tb_importancia,
    file = paste0("impo_", GVEZ, ".txt"),
    sep = "\t"
  )

  GVEZ <<- GVEZ + 1

  umbral <- tb_importancia[
    Feature %like% "canarito",
    median(pos) + canaritos_desvios * sd(pos)
  ] # Atencion corto en la mediana mas desvios!!

  col_utiles <- tb_importancia[
    pos < umbral & !(Feature %like% "canarito"),
    Feature
  ]

  col_utiles <- unique(c(
    col_utiles,
    c(campitos, "mes")
  ))

  col_inutiles <- setdiff(colnames(dataset), col_utiles)

  dataset[, (col_inutiles) := NULL]

  cat( "fin CanaritosAsesinos()\n")

  return( tb_importancia )
}


# Estos DOS parametros son los que se deben modificar
PARAM$CN$ratio <- 0.1 # baje a 0.1 de 0.2
PARAM$CN$desvios <- 2


# Parametros quasi fijos
# Parametros de un LightGBM que se genera para estimar la column importance
PARAM$CN$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$CN$train$positivos <- c( "BAJA+2")
PARAM$CN$train$training <- c( 202101, 202102, 202103)
PARAM$CN$train$validation <- c( 202105 )
PARAM$CN$train$undersampling <- 0.1
PARAM$CN$train$gan1 <- 117000
PARAM$CN$train$gan0 <-  -3000

# la llamada a Canaritos Asesinos
tb_importancia <- CanaritosAsesinos(
  canaritos_ratio = PARAM$CN$ratio,
  canaritos_desvios = PARAM$CN$desvios,
  canaritos_semilla = PARAM$semilla_primigenia
)


# grabo la importancia, ver el archivo directamente en la carpeta

fwrite( tb_importancia,
  file="canaritos.txt",
  sep="\t"
)

# verifico
ncol(dataset)
colnames(dataset)

inicio CanaritosAsesinos()
fin CanaritosAsesinos()


[1] 234

[1] "numero_de_cliente"                                   
  [2] "foto_mes"                                            
  [3] "cliente_edad"                                        
  [4] "mrentabilidad"                                       
  [5] "mrentabilidad_annual"                                
  [6] "mcomisiones"                                         
  [7] "mactivos_margen"                                     
  [8] "mpasivos_margen"                                     
  [9] "cproductos"                                          
 [10] "mcaja_ahorro"                                        
 [11] "cdescubierto_preacordado"                            
 [12] "mcuentas_saldo"                                      
 [13] "ctarjeta_visa_transacciones"                         
 [14] "mtarjeta_visa_consumo"                               
 [15] "mtarjeta_master_consumo"                             
 [16] "mprestamos_personales"                               
 [17] "cpayroll_trx"                                        
 [18] "ccallcenter_transacciones"                           
 [19] "chomebanking_transacciones"                          
 [20] "ctrx_quarter"                                        
 [21] "Master_status"                                       
 [22] "Master_fechaalta"                                    
 [23] "Master_mpagominimo"                                  
 [24] "Visa_status"                                         
 [25] "Visa_fechaalta"                                      
 [26] "Visa_mpagominimo"                                    
 [27] "clase_ternaria"                                      
 [28] "mpayroll_sobre_edad"                                 
 [29] "cliente_edad_lag1"                                   
 [30] "cliente_antiguedad_lag1"                             
 [31] "mrentabilidad_lag1"                                  
 [32] "mrentabilidad_annual_lag1"                           
 [33] "mcomisiones_lag1"                                    
 [34] "mactivos_margen_lag1"                                
 [35] "mpasivos_margen_lag1"                                
 [36] "mcuenta_corriente_lag1"                              
 [37] "mcuentas_saldo_lag1"                                 
 [38] "ctarjeta_visa_transacciones_lag1"                    
 [39] "mtarjeta_visa_consumo_lag1"                          
 [40] "chomebanking_transacciones_lag1"                     
 [41] "ctrx_quarter_lag1"                                   
 [42] "Master_fechaalta_lag1"                               
 [43] "Visa_fechaalta_lag1"                                 
 [44] "Visa_mpagominimo_lag1"                               
 [45] "cliente_edad_lag2"                                   
 [46] "mrentabilidad_lag2"                                  
 [47] "mrentabilidad_annual_lag2"                           
 [48] "mcomisiones_lag2"                                    
 [49] "mactivos_margen_lag2"                                
 [50] "mpasivos_margen_lag2"                                
 [51] "cproductos_lag2"                                     
 [52] "mcaja_ahorro_lag2"                                   
 [53] "ctarjeta_visa_transacciones_lag2"                    
 [54] "mpayroll_lag2"                                       
 [55] "chomebanking_transacciones_lag2"                     
 [56] "ctrx_quarter_lag2"                                   
 [57] "Master_mpagominimo_lag2"                             
 [58] "Visa_fechaalta_lag2"                                 
 [59] "Visa_mpagominimo_lag2"                               
 [60] "mrentabilidad_delta1"                                
 [61] "mrentabilidad_annual_delta1"                         
 [62] "mrentabilidad_annual_delta2"                         
 [63] "mcomisiones_delta1"                                  
 [64] "mcomisiones_delta2"                                  
 [65] "mactivos_margen_delta1"                              
 [66] "mpasivos_margen_delta1"       

### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
PARAM$trainingstrategy$validate <- c(202106, 202107)

PARAM$trainingstrategy$training <- c(
  202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 0.5


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
# solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]


# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]

# -----------------------------------------------------------------------------
# ESTRATEGIA DE PESOS POR TRIMESTRE (OPTIMIZACIÓN BAYESIANA)
# -----------------------------------------------------------------------------
# 1. Crear una columna que identifique el trimestre (ej. 2020Q2)
dataset[, anio := as.integer(substr(as.character(foto_mes), 1, 4))]
dataset[, mes := as.integer(substr(as.character(foto_mes), 5, 6))]

# Calcula el trimestre (1, 2, 3, o 4)
dataset[, trimestre := ceiling(mes / 3)]

# Combina Año y Trimestre para el ranking
dataset[, anio_trimestre := paste0(anio, "Q", trimestre)]

# 2. Obtener los valores únicos de trimestre ordenados
trimestres_unicos <- sort(unique(dataset$anio_trimestre))

# 3. Asignar peso: el trimestre más viejo tiene el peso 1, el siguiente 2, etc.
dataset[, peso_norm := match(anio_trimestre, trimestres_unicos)]


# 4. Verificar la asignación (¡AHORA VA AQUÍ!)
print("Distribución de pesos por trimestre en Optimización Bayesiana:")
print(dataset[, .(peso = unique(peso_norm)), by = anio_trimestre][order(anio_trimestre)])


# 5. Limpiar columnas temporales (opcional, pero buena práctica)
# NOTA: Ahora anio_trimestre ya se usó para la verificación, así que se puede eliminar
dataset[, c("anio", "mes", "trimestre", "anio_trimestre") := NULL]
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  weight= dataset[fold_train == TRUE, peso_norm],  # ¡Peso por Trimestre!
  free_raw_data= TRUE
)

[1] "Distribución de pesos por trimestre en Optimización Bayesiana:"
   anio_trimestre  peso
           <char> <int>
1:         2020Q2     1
2:         2020Q3     2
3:         2020Q4     3
4:         2021Q1     4
5:         2021Q2     5
6:         2021Q3     6


In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘parallelMap’


Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.01,
  feature_fraction= 0.5,
  num_iterations= 5000,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 500
)

# --- 1. Definir los Hiperparámetros para el Modelo Final (SALTÁNDOSE MBO) ---
# Usamos valores fijos en lugar de los obtenidos por optimización
PARAM$out$lgbm$mejores_hiperparametros <- list(
  num_leaves = 8168L, # Valor por defecto común para LGBM
  min_data_in_leaf = 4L
)

# --- 2. Ensamblar el Conjunto de Parámetros Finales ---
# Combina los parámetros fijos (como learning_rate, early_stopping) con los "mejores"
param_final <- modifyList(
  PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
   weight= dataset[fold_final_train == TRUE, peso_norm],  # ¡Peso por Trimestre!
  free_raw_data= TRUE
)
#dtrain <- lgb.Dataset(
 # data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  #label= dataset[fold_train == TRUE, clase01],
 # weight= dataset[fold_train == TRUE, peso_norm],  # ¡Peso por Trimestre!
 # free_raw_data= TRUE
nrow( dfinal_train) # verifico el tamaño

[1] 240562

##### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

#tb_importancia <- as.data.table(lgb.importance(final_model))
#archivo_importancia <- "impo.txt"

#fwrite( tb_importancia,
#  file= archivo_importancia,
 # sep= "\t"
#)

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_800.csv -m 'envios=800  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_850.csv -m 'envios=850  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_900.csv -m 'envios=900  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_950.csv -m 'envios=950  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_1000.csv -m 'envios=1000  semilla=999983'' had status 1”


...

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_1100.csv -m 'envios=1100  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_1150.csv -m 'envios=1150  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_1200.csv -m 'envios=1200  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_1250.csv -m 'envios=1250  semilla=999983'' had status 1”


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA67265_1300.csv -m 'envios=1300  semilla=999983'' had status 1”


In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 09:14:27 PM 2025"